In [1]:
pip install ktrain

In [2]:
import numpy as np
import os

import tensorflow as tf
import ktrain
from ktrain import text
import pandas as pd

In [3]:
df = pd.read_csv('../input/articles/articles.csv')

In [4]:
import nltk
nltk.download('stopwords')

In [5]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [6]:
df.head()

In [7]:
df['Text'] = df['Text'].astype('str')
df['without_stopwords'] = df['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [8]:
df.shape[0]*0.8

In [9]:
data = df.sample(n = 1192, random_state = 2)

In [10]:
data['Category'].unique()

In [11]:
df_test = df[['without_stopwords','Category']]

In [12]:
df_test = df_test[~df_test.without_stopwords.isin(data.without_stopwords)]

In [13]:
df_test.shape

In [14]:
(X_train,y_train),(X_test,y_test),preproc = text.texts_from_df(train_df=data, text_column = 'without_stopwords',
                                                               label_columns = ['Category'],
                                                               val_df = df_test,
                                                               maxlen = 500,
                                                               preprocess_mode = 'bert')

In [15]:
model = text.text_classifier(name = 'bert',
                             train_data = (X_train,y_train),
                             preproc = preproc)

In [16]:
learner = ktrain.get_learner(model = model,
                             train_data= (X_train,y_train),
                             val_data = (X_test,y_test),
                            batch_size = 6)

In [17]:
learner.fit_onecycle(lr = 2e-5,
                     epochs = 3)

In [18]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save('email_classifier')
print('MODEL SAVED')

In [19]:
df_test['without_stopwords'] = df_test['without_stopwords'].astype('str')
df_test['predicted_class'] = [predictor.predict(x) for x in df_test['without_stopwords']]
df_test['predicted_prob'] = [predictor.predict(x,return_proba = 'True') for x in df_test['without_stopwords']]

In [21]:
df_test